[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/chatbots/nemo-guardrails/01-variables-and-flows.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/chatbots/nemo-guardrails/01-variables-and-flows.ipynb)

# Nemo Guardrails - Variables and Flows

In [ ]:
### In case you encounter the error run the cell twice in google colab to resolve

!pip install -U nemoguardrails langchain langchain-community langchain-groq groq


  Using cached langchain-1.0.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_groq-1.0.0-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.


In [ ]:
### Below is the exact version of packages to be installed

# !pip install -U nemoguardrails==0.18.0 langchain==0.3.27 langchain-community==0.3.31 langchain-groq==0.3.8 groq==0.33.0

# Variables and Flows


Colang is the modelling language used to implement conversational flows to build chatbots by Nvidia & is used by Nemo guardrails library

Colang also allows us to use logical flows (if-else) and insert variables into our rails. There are several ways of inputing variables. When inserting the full conversational memory into our rails (rather than a single `prompt` as we have been doing so far) we can use the `context` message to initialize variables.

Let's begin by doing that. First, we'll initialize a simple Rail.

We need to set our Groq API key. Also set the Hugging Face token 'HF_TOKEN' in secrets section

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')



In [ ]:
colang_content = """
define user greeting
    "Hey there!"
    "How are you?"
    "What's up?"

define bot greeting
    "Hey there!"

define bot name greeting
    "Hey $name!"

define flow
    user greeting
    if $name
        bot name greeting
    else
        bot greeting
"""
yaml_content = """
models:
- type: main
  engine: groq
  model: llama-3.1-8b-instant
"""

Initialize the Rail:

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)
# create rails
rails = LLMRails(config)

Now, to begin using the `messages` input we need to set a list of messages which must be structured as a list of dictionary objects each containing `"role"` and `"content"` keys. Guardrails allows us to use three roles, `"context"`, `"user"`, and `"assistant"`.

In [ ]:
### The content key is used to pass the variables

messages = [
    {"role": "context", "content": ""},
    {"role": "user", "content": "Hey there!"}
    #{"role": "assistant",
    # "content": "Hi! How are you? Is there anything I can help with?"},
]

In [ ]:
### By passing messages we can pass conversational history with variables

await rails.generate_async(messages=messages)

{'role': 'assistant',
 'content': 'It appears that the current conversation is stuck in a loop. The user sent a greeting message "Hey there!", and the bot responded with a general response, but didn\'t actually match the user\'s intent, which seems to be "Hey there!" (i.e., another greeting).\nTo proceed, it would be best to update the bot\'s intent to match the user\'s intent, which would be to express a greeting. Here\'s an updated version of the conversation:'}

Right now the bot doesn't greet us with our name because we left the `context` empty. Instead, let's try passing in the `$name` parameter.

In [ ]:
messages = [
    {"role": "context", "content": {"name": "Ankur"}},
    {"role": "user","content": "Hi!"},

]

In [ ]:
# Ask Guardrails to also return debug info so we can see if the flow triggers
options = {"log": {"activated_rails": True}}

In [ ]:
result = rails.generate_async(messages=messages, options = options)

In [ ]:
rails.generate_async(messages=messages)

<coroutine object LLMRails.generate_async at 0x14c36bb0>

In [ ]:

await rails.generate_async(messages=messages)

{'role': 'assistant',
 'content': 'Here is a possible continuation of the conversation between the user and the bot:'}

Above we're using the `context` to set context variables within our Rails. Sometimes we may want to set them from within a conversation. Let's see how we could grab the users name and pass it into a `$name` variable.

In [ ]:
colang_content = """
define user give name
    "My name is James"
    "I'm Julio"
    "Sono Andrea"

define user greeting
    "Hey there!"
    "How are you?"
    "What's up?"

define bot name greeting
    "Hey $name!"

define flow give name
    user give name
    $name = ...
    bot name greeting

define flow
    user greeting
    if not $name
        bot ask name
    else
        bot name greeting
"""

Remove `$name` parameter from the `context`:

In [ ]:
messages = [
    {"role": "context", "content": ""},
    {"role": "assistant",
     "content": "Hi! How are you? Is there anything I can help with?"},
    {"role": "user", "content": "Hey there!"}
]

In [ ]:
# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)
# create rails
rails = LLMRails(config)

In [ ]:
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': "Hi there! What's your name?"}

In [ ]:
messages += [
    res,
    {"role": "user", "content": "I'm James"}
]
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': 'Hey James!'}

---